In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import pandas as pd

def read_json_to_dataframe(file_path):
    # Read the JSON file line by line
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    return df

# Read input data
input_file_path = '/kaggle/input/datahon/attribute_train.data'
input_df = read_json_to_dataframe(input_file_path)

# Read output data
output_file_path = '/kaggle/input/datahon/attribute_train.solution'
output_df = read_json_to_dataframe(output_file_path)

# Check the number of rows in each DataFrame
print(f"\nNumber of rows in input DataFrame: {len(input_df)}")
print(f"Number of rows in output DataFrame: {len(output_df)}")

# Merge the input and output DataFrames on 'indoml_id'
merged_df = pd.merge(input_df, output_df, on='indoml_id', how='inner')

print("\nMerged DataFrame:")
print(merged_df.head())
print(f"\nNumber of rows in merged DataFrame: {len(merged_df)}")

# Save the merged DataFrame to a CSV file (optional)
merged_df.to_csv('merged_data.csv', index=False)

In [ ]:
merged_df

In [ ]:
import pandas as pd
from collections import Counter

# Assuming merged_df is our DataFrame from the previous step

# Function to count unique values and print the top N
def analyze_column(df, column_name, top_n=10):
    value_counts = df[column_name].value_counts()
    total_values = len(value_counts)
    print(f"\n{column_name}:")
    print(f"Total unique values: {total_values}")
    print(f"Top {top_n} most common values:")
    print(value_counts.head(top_n))
    return total_values

# Analyze each category level and brand
num_brands = analyze_column(merged_df, 'details_Brand')
num_l0 = analyze_column(merged_df, 'L0_category')
num_l1 = analyze_column(merged_df, 'L1_category')
num_l2 = analyze_column(merged_df, 'L2_category')
num_l3 = analyze_column(merged_df, 'L3_category')
num_l4 = analyze_column(merged_df, 'L4_category')



In [ ]:
merged_df.rename(columns={'details_Brand': 'L5_category'}, inplace=True)

In [ ]:
merged_df.head()

In [ ]:
import json
import pandas as pd

def read_json_to_dataframe(file_path):
    # Read the JSON file line by line
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    return df

# Read input data
input_file_path = '/kaggle/input/datahon/attribute_val.data'
input_df = read_json_to_dataframe(input_file_path)

# Read output data
output_file_path = '/kaggle/input/datahon/attribute_val.solution'
output_df = read_json_to_dataframe(output_file_path)



# Check the number of rows in each DataFrame
print(f"\nNumber of rows in input DataFrame: {len(input_df)}")
print(f"Number of rows in output DataFrame: {len(output_df)}")

# Merge the input and output DataFrames on 'indoml_id'
merged_val_df = pd.merge(input_df, output_df, on='indoml_id', how='inner')

print("\nMerged DataFrame:")
print(merged_val_df.head())
print(f"\nNumber of rows in merged DataFrame: {len(merged_df)}")

# Save the merged DataFrame to a CSV file (optional)
merged_val_df.to_csv('merged_data.csv', index=False)

In [ ]:
merged_val_df.rename(columns={'details_Brand': 'L5_category'}, inplace=True)

In [ ]:
merged_val_df.head()

In [ ]:
print(set(merged_df['L0_category'].to_list()) == set(merged_val_df['L0_category'].to_list()))
print(set(merged_df['L1_category'].to_list()) == set(merged_val_df['L1_category'].to_list()))
print(set(merged_df['L2_category'].to_list()) == set(merged_val_df['L2_category'].to_list()))
print(set(merged_df['L3_category'].to_list()) == set(merged_val_df['L3_category'].to_list()))
print(set(merged_df['L4_category'].to_list()) == set(merged_val_df['L4_category'].to_list()))

<h1>TODO: Remove</h1>

In [ ]:
# merged_val_df = merged_val_df[:5]
# merged_df = merged_df[:5]

In [ ]:
import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaModel
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import pandas as pd
import numpy as np

class HierarchicalProductClassifier(nn.Module):
    def __init__(self, num_categories, embedding_dim=768):
        super(HierarchicalProductClassifier, self).__init__()
        self.roberta = RobertaModel.from_pretrained('distilroberta-base')  # Use a less computationally expensive model
        self.dropout = nn.Dropout(0.1)
        
        self.category_classifiers = nn.ModuleList([
            nn.Linear(embedding_dim, num_categories) for _ in range(6)
        ])

    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = roberta_output.pooler_output
        pooled_output = self.dropout(pooled_output)
        
        category_logits = [classifier(pooled_output) for classifier in self.category_classifiers]
        
        return category_logits


In [ ]:
class ProductDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.input_ids = encodings['input_ids']
        self.attention_mask = encodings['attention_mask']
        self.category_labels = [encodings['category_l0'],encodings['category_l1'],encodings['category_l2'],encodings['category_l3'],encodings['category_l4'],encodings['category_l5']]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        item = {
           'input_ids': self.input_ids[idx].clone().detach(),
        'attention_mask': self.attention_mask[idx].clone().detach(),
        }
        item.update({f'category_l{i}': self.category_labels[i][idx].clone().detach() for i in range(6)})
        return item


In [ ]:
def preprocess_data(df, tokenizer, category_encoder):
    texts = df['title'].tolist()
    categories = [df[f'L{i}_category'].tolist() for i in range(6)]

    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
    category_ids = [torch.tensor(category_encoder.transform(cats), dtype=torch.long) for cats in categories]

    return {
        'input_ids': torch.tensor(encodings['input_ids'], dtype=torch.long),
        'attention_mask': torch.tensor(encodings['attention_mask'], dtype=torch.long),
        'category_l0': category_ids[0],
        'category_l1': category_ids[1],
        'category_l2': category_ids[2],
        'category_l3': category_ids[3],
        'category_l4': category_ids[4],
        'category_l5': category_ids[5],
    }

In [ ]:

from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.utils as utils
import torch.nn as nn
import copy

def compute_metrics(predictions, labels):
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return accuracy, f1

def evaluate(predictions, labels):
    overall_accuracy = 0
    overall_f1 = 0
    num_tasks = len(predictions)

    print("\nEvaluation Metrics:")
    for key in predictions.keys():
        accuracy, f1 = compute_metrics(predictions[key], labels[key])
        print(f"{key.capitalize()}:")
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  F1 Score: {f1:.4f}")
        overall_accuracy += accuracy
        overall_f1 += f1

    print("\nOverall Metrics:")
    print(f"  Average Accuracy: {overall_accuracy / num_tasks:.4f}")
    print(f"  Average F1 Score: {overall_f1 / num_tasks:.4f}")
    return overall_accuracy / num_tasks    

In [ ]:
def train(model, train_dataloader, val_dataloader, optimizer, device, num_epochs, category_encoder, patience=3):
    best_accuracy = 0
    best_model_wts = copy.deepcopy(model.state_dict())
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True)
    early_stopping_patience = patience
    early_stopping_counter = 0
    loss_fct = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        train_predictions = {f'category_l{i}': [] for i in range(6)}
        train_labels = {f'category_l{i}': [] for i in range(6)}

        for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            category_labels = [batch[f'category_l{i}'].to(device) for i in range(6)]
            category_logits = model(input_ids, attention_mask)

            loss = sum(loss_fct(logits, labels) for logits, labels in zip(category_logits, category_labels))
            loss.backward()

            # Gradient clipping
            utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()

            # Store predictions and labels for later evaluation
            for i, logits in enumerate(category_logits):
                train_predictions[f'category_l{i}'].extend(torch.argmax(logits, dim=1).cpu().numpy())
                train_labels[f'category_l{i}'].extend(category_labels[i].cpu().numpy())

        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

        # Use stored predictions and labels for evaluation on the training set
        print("\nTrain Set Evaluation:")
        evaluate(train_predictions, train_labels)

        # Evaluate on the validation set
        val_predictions, val_labels, val_loss = evaluate_dataloader(model, val_dataloader, device, compute_loss=True)
        print("\nValidation Set Evaluation:")
        evaluate(val_predictions, val_labels)
        print(f"Validation Loss: {val_loss:.4f}")

        # Scheduler step
        accuracy = evaluate(val_predictions, val_labels)
        scheduler.step(accuracy)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_wts = copy.deepcopy(model.state_dict())
            early_stopping_counter = 0  # Reset the early stopping counter
        else:
            early_stopping_counter += 1

        # Early stopping
        if early_stopping_counter >= early_stopping_patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

    model.load_state_dict(best_model_wts)  # Load the best model weights
    return best_accuracy

def evaluate_dataloader(model, dataloader, device, compute_loss=False):
    model.eval()
    all_predictions = {f'category_l{i}': [] for i in range(6)}
    all_labels = {f'category_l{i}': [] for i in range(6)}
    total_loss = 0
    loss_fct = nn.CrossEntropyLoss()

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            category_logits = model(input_ids, attention_mask)

            if compute_loss:
                category_labels = [batch[f'category_l{i}'].to(device) for i in range(6)]
                loss = sum(loss_fct(logits, labels) for logits, labels in zip(category_logits, category_labels))
                total_loss += loss.item()

            for i, logits in enumerate(category_logits):
                all_predictions[f'category_l{i}'].extend(torch.argmax(logits, dim=1).cpu().numpy())
                all_labels[f'category_l{i}'].extend(batch[f'category_l{i}'].cpu().numpy())

    if compute_loss:
        avg_loss = total_loss / len(dataloader)
        return all_predictions, all_labels, avg_loss
    else:
        return all_predictions, all_labels


In [ ]:
# Load data
train_df = merged_df
val_df = merged_val_df
# Create label encoders
category_encoder = LabelEncoder()

all_categories = []
for df in [train_df, val_df]:
    for i in range(6):
        all_categories.extend(df[f'L{i}_category'].tolist())

category_encoder.fit(all_categories)

tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')

train_dataset = ProductDataset(preprocess_data(train_df, tokenizer, category_encoder))
val_dataset = ProductDataset(preprocess_data(val_df, tokenizer, category_encoder))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalProductClassifier(len(category_encoder.classes_))
model = nn.DataParallel(model)
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
num_epochs = 11
print("Starting model training...")
best_accuracy = train(model, train_dataloader, val_dataloader, optimizer, device, num_epochs, category_encoder)

print(f"Training completed. Best validation accuracy: {best_accuracy:.4f}")


Here below there is a new approach used added early stopping reduce lr on pleateu and embeddings for store_name and manufracturer name

In [ ]:
import json
import pandas as pd

def read_json_to_dataframe(file_path):
    # Read the JSON file line by line
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    return df

# Read input data
input_file_path = '/kaggle/input/datahon/attribute_test.data'
test_df = read_json_to_dataframe(input_file_path)
test_df.rename(columns={'details_Brand': 'L5_category'}, inplace=True)

<h1>TODO: Remove</h1>

In [ ]:
# test_df = test_df[:5]

In [ ]:
from tqdm.notebook import tqdm  # or just `from tqdm import tqdm` if not using Jupyter

def preprocess_test_data(df, tokenizer):
    texts = df['title'].tolist()
    
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
    
    return {
        'input_ids': torch.tensor(encodings['input_ids'], dtype=torch.long),
        'attention_mask': torch.tensor(encodings['attention_mask'], dtype=torch.long)
    }

class ProductTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.input_ids = encodings['input_ids']
        self.attention_mask = encodings['attention_mask']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx].clone().detach(),
        'attention_mask': self.attention_mask[idx].clone().detach(),
        }

# 2. Preprocess the test data
test_encodings = preprocess_test_data(test_df, tokenizer)
test_dataset = ProductTestDataset(test_encodings)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. Make predictions
model.eval()
all_predictions = {f'raw_L{i}_category': [] for i in range(6)}
decoded_predictions = {f'L{i}_category': [] for i in range(6)}

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Predicting", leave=False):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        category_logits = model(input_ids, attention_mask)

        for i, logits in enumerate(category_logits):
            argmax_values = torch.argmax(logits, dim=1).cpu().numpy()
            all_predictions[f'raw_L{i}_category'].extend(argmax_values)
            decoded_values = category_encoder.inverse_transform(argmax_values)
            decoded_predictions[f'L{i}_category'].extend(decoded_values)

# 4. Create DataFrame with raw and decoded predictions
predictions_df = pd.DataFrame({**all_predictions, **decoded_predictions})

# 5. Merge predictions with original test_df
merged_df = pd.concat([test_df.reset_index(drop=True), predictions_df], axis=1)

# 6. Save the merged DataFrame to a CSV file
merged_df.rename(columns={'L5_category': 'details_Brand'}, inplace=True)
merged_df.to_csv('test_predictions_with_raw_and_labels.csv', index=False)

print("Predictions (raw and decoded) saved to 'test_predictions_with_raw_and_labels.csv'.")


In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
# import torch
# import torch.nn as nn
# from transformers import RobertaTokenizer, RobertaModel, get_linear_schedule_with_warmup
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import accuracy_score, f1_score
# from tqdm import tqdm
# import pandas as pd
# import numpy as np
# # import nlpaug.augmenter.word as naw

# class HierarchicalProductClassifier(nn.Module):
#     def __init__(self, num_categories, num_stores, num_manufacturers, embedding_dim=768):
#         super(HierarchicalProductClassifier, self).__init__()
#         self.roberta = RobertaModel.from_pretrained('distilroberta-base')
#         self.dropout = nn.Dropout(0.1)
#         self.store_embedding = nn.Embedding(num_stores, embedding_dim)
#         self.manufacturer_embedding = nn.Embedding(num_manufacturers, embedding_dim)
        
#         self.category_classifiers = nn.ModuleList([
#             nn.Linear(embedding_dim * 3, num_categories) for _ in range(5)
#         ])

#     def forward(self, input_ids, attention_mask, store_ids, manufacturer_ids):
#         roberta_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
#         pooled_output = roberta_output.pooler_output
#         pooled_output = self.dropout(pooled_output)
        
#         store_emb = self.store_embedding(store_ids)
#         manufacturer_emb = self.manufacturer_embedding(manufacturer_ids)
        
#         combined = torch.cat((pooled_output, store_emb, manufacturer_emb), dim=1)
        
#         category_logits = [classifier(combined) for classifier in self.category_classifiers]
        
#         return category_logits

# class ProductDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.input_ids = encodings['input_ids']
#         self.attention_mask = encodings['attention_mask']
#         self.store_ids = encodings['store_ids']
#         self.manufacturer_ids = encodings['manufacturer_ids']
#         self.category_labels = [encodings[f'category_l{i}'] for i in range(5)]

#     def __len__(self):
#         return len(self.input_ids)

#     def __getitem__(self, idx):
#         item = {
#             'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
#             'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
#             'store_ids': torch.tensor(self.store_ids[idx], dtype=torch.long),
#             'manufacturer_ids': torch.tensor(self.manufacturer_ids[idx], dtype=torch.long),
#         }
#         item.update({f'category_l{i}': torch.tensor(self.category_labels[i][idx], dtype=torch.long) for i in range(5)})
#         return item

# def preprocess_data(df, tokenizer, category_encoder, store_encoder, manufacturer_encoder, aug=None):
#     texts = df['title'].tolist()
#     if aug:
#         texts = [aug.augment(text)[0] for text in texts]
#     stores = df['store'].tolist()
#     manufacturers = df['details_Manufacturer'].tolist()
#     categories = [df[f'L{i}_category'].tolist() for i in range(5)]

#     encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
#     store_ids = store_encoder.transform(stores)
#     manufacturer_ids = manufacturer_encoder.transform(manufacturers)
#     category_ids = [category_encoder.transform(cats) for cats in categories]

#     return {
#         'input_ids': encodings['input_ids'],
#         'attention_mask': encodings['attention_mask'],
#         'store_ids': store_ids,
#         'manufacturer_ids': manufacturer_ids,
#         'category_l0': category_ids[0],
#         'category_l1': category_ids[1],
#         'category_l2': category_ids[2],
#         'category_l3': category_ids[3],
#         'category_l4': category_ids[4],
#     }

# def compute_metrics(predictions, labels):
#     accuracy = accuracy_score(labels, predictions)
#     f1 = f1_score(labels, predictions, average='weighted')
#     return accuracy, f1

# def evaluate(model, dataloader, device, category_encoder):
#     model.eval()
#     all_predictions = {f'category_l{i}': [] for i in range(5)}
#     all_labels = {f'category_l{i}': [] for i in range(5)}
    
#     with torch.no_grad():
#         for batch in tqdm(dataloader, desc="Evaluating"):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             store_ids = batch['store_ids'].to(device)
#             manufacturer_ids = batch['manufacturer_ids'].to(device)
            
#             category_logits = model(input_ids, attention_mask, store_ids, manufacturer_ids)
            
#             for i, logits in enumerate(category_logits):
#                 all_predictions[f'category_l{i}'].extend(torch.argmax(logits, dim=1).cpu().numpy())
#                 all_labels[f'category_l{i}'].extend(batch[f'category_l{i}'].cpu().numpy())
    
#     print("\nEvaluation Metrics:")
#     overall_accuracy = 0
#     overall_f1 = 0
#     for key in all_predictions.keys():
#         predictions = category_encoder.inverse_transform(all_predictions[key])
#         labels = category_encoder.inverse_transform(all_labels[key])
        
#         accuracy, f1 = compute_metrics(predictions, labels)
#         print(f"{key.capitalize()}:")
#         print(f"  Accuracy: {accuracy:.4f}")
#         print(f"  F1 Score: {f1:.4f}")
#         overall_accuracy += accuracy
#         overall_f1 += f1
    
#     num_tasks = len(all_predictions)
#     print("\nOverall Metrics:")
#     print(f"  Average Accuracy: {overall_accuracy/num_tasks:.4f}")
#     print(f"  Average F1 Score: {overall_f1/num_tasks:.4f}")
    
#     return overall_accuracy / num_tasks

# def train(model, train_dataloader, val_dataloader, optimizer, scheduler, device, num_epochs, category_encoder):
#     best_accuracy = 0
#     patience = 3
#     no_improve_epochs = 0

#     for epoch in range(num_epochs):
#         model.train()
#         total_loss = 0
#         for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
#             optimizer.zero_grad()
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             store_ids = batch['store_ids'].to(device)
#             manufacturer_ids = batch['manufacturer_ids'].to(device)
#             category_labels = [batch[f'category_l{i}'].to(device) for i in range(5)]

#             category_logits = model(input_ids, attention_mask, store_ids, manufacturer_ids)

#             loss_fct = nn.CrossEntropyLoss()
#             loss = sum(loss_fct(logits, labels) for logits, labels in zip(category_logits, category_labels))
#             loss.backward()

#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#             optimizer.step()
#             scheduler.step()

#             total_loss += loss.item()
        
#         avg_loss = total_loss / len(train_dataloader)
#         print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")
#         print("\nEvaluating on train set...")
#         accuracy = evaluate(model, train_dataloader, device, category_encoder)
        
        
#         print("\nEvaluating on validation set...")
#         accuracy = evaluate(model, val_dataloader, device, category_encoder)
        
#         if accuracy > best_accuracy:
#             best_accuracy = accuracy
#             no_improve_epochs = 0
#             torch.save(model.state_dict(), 'best_model.pth')
#         else:
#             no_improve_epochs += 1
        
#         if no_improve_epochs >= patience:
#             print(f"Early stopping triggered after epoch {epoch+1}")
#             break
    
#     return best_accuracy

# # Load data
# train_df = merged_df[:10000]
# val_df = merged_val_df[:10000]

# # Create label encoders
# category_encoder = LabelEncoder()
# store_encoder = LabelEncoder()
# manufacturer_encoder = LabelEncoder()

# all_categories = []
# all_stores = train_df['store'].tolist() + val_df['store'].tolist()
# all_manufacturers = train_df['details_Manufacturer'].tolist() + val_df['details_Manufacturer'].tolist()

# for df in [train_df, val_df]:
#     for i in range(5):
#         all_categories.extend(df[f'L{i}_category'].tolist())

# category_encoder.fit(all_categories)
# store_encoder.fit(all_stores)
# manufacturer_encoder.fit(all_manufacturers)

# tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')

# # Data augmentation
# # aug = naw.SynonymAug(aug_src='wordnet')

# train_dataset = ProductDataset(preprocess_data(train_df, tokenizer, category_encoder, store_encoder, manufacturer_encoder))
# val_dataset = ProductDataset(preprocess_data(val_df, tokenizer, category_encoder, store_encoder, manufacturer_encoder))

# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = HierarchicalProductClassifier(len(category_encoder.classes_), len(store_encoder.classes_), len(manufacturer_encoder.classes_)).to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# num_training_steps = len(train_dataloader) * num_epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# num_epochs = 20
# print("Starting model training...")
# best_accuracy = train(model, train_dataloader, val_dataloader, optimizer, scheduler, device, num_epochs, category_encoder)
# print(f"Training completed. Best validation accuracy: {best_accuracy:.4f}")

new model has not been able to beat old one despite adding the complexity